In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install hazm
!pip install --upgrade git+https://github.com/GRAAL-Research/poutyne.git@dev #install poutyne
!pip install --upgrade colorama #install colorama
!pip install --upgrade pymagnitude-light #install pymagnitude-light
!matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=52c2d3b56e41f3606802445b5431ea967c9c427e83c0407f827a7dbe76acef6a
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180386 sha256=b90cfdcd879d411b6a075ce7c02099aaa9d4a8ceb7e6ec85d5908191e1521999
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
در این قسمت، داده‌های برچسب گذاری شده را ابتدا نرمال سازی می‌کنیم و بعد توکن‌ها 
را به دست می‌آوریم و آن را به فرمتی که برای تزریق به شبکه مورد نیاز است در می‌آوریم.
</div>

In [2]:
import codecs
import math
import numpy as np
import tqdm
import torch
from hazm import *

class Loader:
    path_data = '/content/drive/My Drive/all.samples.out.txt'
    convert_tags = {'O': '<n>', 'B-علت'
    : '<c>', 'I-علت'
    : '<c>', 'B-معلول'
    : '<e>', 'I-معلول'
    : '<e>', 'B-نشانه'
    : '<m>', 'I-نشانه'
    : '<m>'}

    def __init__(self):
        self.normalizer = Normalizer()
        self.lemmatizer = Lemmatizer()

    def create_data_label(self):
        lines = codecs.open(self.path_data, 'r', 'utf-8').readlines()

        token_list = []
        label_list = []
        data = []
        line_number = 1
        for l in lines:
            line_number += 1
            if l in ['\n']:
                if len(token_list) > 0:
                    
                    tokenToStr = ' '.join([str(elem) for elem in token_list])
                    token = (tokenToStr, label_list)
                    if len(tokenToStr.split()) == len(label_list) : 
                     data.append(token)
                    token_list = []
                    label_list = []
                continue

            l = l[0:len(l) - 1]
            w = l.split('\t')
            if len(w) != 2:
                continue

            word = w[0]
            label = w[1]
            word = self.normalizer.normalize(word)
            word = self.lemmatizer.lemmatize(word)
            token_list.append(word)
            try:
                label_list.append(self.convert_tags[label])
            except:
                label_list.append(self.convert_tags['O'])
                print(f"An exception occurred for label = {label} in line_number = {line_number}")


        return data

loader = Loader()
data = loader.create_data_label()
print(f'Data[0] = {data[0]}')


An exception occurred for label = IO in line_number = 134549
Data[0] = ('این مساله باعث شد#شو او را از بانفوذ اعضا جوان حزب ندا ایران در میان اصلاح طلب شناخت#شناس .', ['<c>', '<c>', '<m>', '<n>', '<n>', '<n>', '<n>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>'])


In [3]:
import gzip
import shutil
import warnings

import torch
import torch.nn as nn
import torch.optim as optim
from poutyne import set_seeds
from poutyne.framework import Experiment
from pymagnitudelight import Magnitude
from torch.nn.functional import cross_entropy
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence
from torch.utils.data import DataLoader

<div style='direction:rtl; font-family: "B Lotus"; font-size: 16px'>
در این قسمت مشخصات شبکه را تنظیم می‌کنیم. از ال‌اس‌تی‌ام یک لایه و یک طرفه استفاده می‌کنیم. خروجی ال‌اس‌تی‌ام ۳۰۰ بعدی است که باید به فضای ۴ بعدی که تعداد تگ‌هاست مپ شود. سپس  تنظیمات طول ایپاک و سایز بچ را انجام می‌دهیم.

In [4]:
dimension = 300
num_layer = 1
bidirectional = False

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

input_dim = dimension 
tag_dimension = 4

fully_connected_network = nn.Linear(input_dim, tag_dimension)

device = torch.device("cuda:0")

batch_size = 64
lr = 0.1

epoch_number = 10
set_seeds(42)

داده‌ها به تست و ارزیابی و آموزش تقسیم می‌شوند و یک نمونه خروجی از هر کدام می‌بینیم.

In [5]:
np.random.shuffle(data)
train_size = math.ceil(len(data) * 0.8)
val_size = math.ceil(len(data) * 0.1)
test_size = len(data) - train_size - val_size

train_data = data[0:train_size]
valid_data = data[train_size:train_size + val_size]
test_data = data[train_size + val_size:train_size + val_size + test_size]
print(train_data[0:1])
print(valid_data[0:1])
print(test_data[0:1])


[('کوتاه شدن تاخیرها و کاهش هزینه های مالی ناشی از جهانی شدن منجر به جابجایی بین المللی آسان تر کالا و افراد شد#شو .', ['<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<m>', '<m>', '<c>', '<c>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>'])]
[('اقدام گذشته آمریکا در تحریم به دلیل منافع راهبردی حاصله برای اقتصاد نظام مقدس جمهوری اسلام ایران از جمله کاهش وابستگی به نفت ، تاکنون اقدامات آمریکا بدون پاسخ مانده بود#باش ( از جمله حفظ ذخایر و فروش نفت به قیمت بالا در دهه آینده ) اما ورود آمریکا به خرید عادی و حتی غذا و دارو برای اتباع ایران و دخالت در روابط ایران با کشور همسایه و شرکا تجار .', ['<n>', '<n>', '<n>', '<n>', '<n>', '<m>', '<m>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<c>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<e>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', '<n>', 

جاسازی توکن‌ها به کمک فست تکست انجام می‌شود و تگ‌ها هم شماره ۰ تا ۳ دریافت می‌کنند..

In [6]:
!pip install --upgrade fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import contextlib
import os
import pickle
import re
import sys
from io import TextIOBase

import fasttext
import fasttext.util
import requests

class LookForProgress(TextIOBase):
    def __init__(self, stdout):
        self.stdout = stdout
        self.regex = re.compile(r'([0-9]+(\.[0-9]+)?%)', re.IGNORECASE)

    def write(self, o):
        res = self.regex.findall(o)
        if len(res) != 0:
            print(f"\r{res[-1][0]}", end='', file=self.stdout)


class EmbeddingVectorizer:
    def __init__(self):
        with contextlib.redirect_stdout(LookForProgress(sys.stdout)):
            fasttext.util.download_model('fa', if_exists='ignore')
        self.embedding_model = fasttext.load_model("./cc.fa.300.bin")

    def __call__(self, sent):

        embeddings = []
        for word in sent.split():
            embeddings.append(self.embedding_model[word])
        return embeddings


embedding_vectorizer = EmbeddingVectorizer()

In [8]:
class DatasetBucket:
    def __init__(self, dataset, embedding_model, predict=False):
        self.data = dataset
        self.embedding_model = embedding_model
        self.predict = predict
        self.tags_set = {
            '<n>': 0,
            '<c>': 1,
            '<m>': 2,
            '<e>': 3,

        }

    def __len__(self):
        # for the dataloader
        return len(self.data)

    def __getitem__(self, item):
        data = self.data[item]

        if not self.predict:
            address = data[0]
            address_vector = self.embedding_model(address)

            tags = data[1]
            idx_tags = self._convert_tags_to_idx(tags)
            return address_vector, idx_tags

        address_vector = self.embedding_model(data)
        return address_vector

    def _convert_tags_to_idx(self, tags):
        idx_tags = []
        for tag in tags:
            idx_tags.append(self.tags_set[tag])
        return idx_tags

train_dataset_vectorizer = DatasetBucket(train_data, embedding_vectorizer)
valid_dataset_vectorizer = DatasetBucket(valid_data, embedding_vectorizer)
test_dataset_vectorizer = DatasetBucket(test_data, embedding_vectorizer)

برای یکسان کردن طول جملات از پدینگ استفاده می‌شود.

In [9]:
def pad_collate_fn(batch):

    sequences_vectors, sequences_labels, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
            for (seq_vectors, labels) in sorted(
                batch, key=lambda x: len(x[0]), reverse=True
            )
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(
        sequences_vectors, batch_first=True, padding_value=0
    )
    pack_padded_sequences_vectors = pack_padded_sequence(
        padded_sequences_vectors, lengths.cpu(), batch_first=True
    )  

    padded_sequences_labels = pad_sequence(
        sequences_labels, batch_first=True, padding_value=-100
    )

    return pack_padded_sequences_vectors, padded_sequences_labels

In [10]:
train_loader = DataLoader(
    train_dataset_vectorizer,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
valid_loader = DataLoader(
    valid_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
test_loader = DataLoader(
    test_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=2,
)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


در این قسمت آموزش مدل انجام می‌شود.

In [11]:
class RecurrentNet(nn.Module):
    def __init__(self, lstm_network, fully_connected_network):
        super().__init__()
        self.hidden_state = None

        self.lstm_network = lstm_network
        self.fully_connected_network = fully_connected_network

    def forward(self, packed_sequences_vectors):
        lstm_out, self.hidden_state = self.lstm_network(packed_sequences_vectors)
        lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True)

        tag_space = self.fully_connected_network(lstm_out)
        return tag_space.transpose(-1, 1)  # We need to transpose since it's a sequence


full_network = RecurrentNet(lstm_network, fully_connected_network)

In [32]:

from poutyne import set_seeds, Model
optimizer = optim.SGD(full_network.parameters(), lr)
loss_function = nn.CrossEntropyLoss()
model = Model(
    full_network,
    optimizer,
    loss_function,
     batch_metrics=['accuracy','f1' ,'recall' ],
    device=device,
)

model.fit_generator(train_loader, valid_loader, epochs=30)

test_loss, test_acc = model.evaluate_generator(test_loader)



/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a 

Epoch:  1/30 Step: 109/110  99.09% |███████████████████▊|ETA: 0.05s loss: 0.868044 acc: 65.018311 fscore_macro: 0.616896 recall_macro: 0.582889

<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Train steps: 110 Val steps: 14 9.72s loss: 0.903043 acc: 63.147550 fscore_macro: 0.558083 recall_macro: 0.518227 val_loss: 0.967101 val_acc: 60.524401 val_fscore_macro: 0.506312 val_recall_macro: 0.467602


<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

...

Epoch: 30/30 Step: 107/110  97.27% |███████████████████▍|ETA: 0.17s loss: 0.857455 acc: 66.221954 fscore_macro: 0.582487 recall_macro: 0.541310

<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch: 30/30 Train steps: 110 Val steps: 14 9.69s loss: 0.822449 acc: 66.005275 fscore_macro: 0.606243 recall_macro: 0.568425 val_loss: 0.846877 val_acc: 64.595933 val_fscore_macro: 0.612506 val_recall_macro: 0.588132


<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Test steps: 14 1.69s test_loss: 0.824796 test_acc: 65.597305 test_fscore_macro: 0.629010 test_recall_macro: 0.605060                                                


روی یک داده‌ی نمونه خروجی می‌گیریم تا تگ‌های تولید شده را مشاهده کنیم.

In [47]:
predict_data = ["راحت به دست آوردن اسلحه منجر به نرخ بالای مرگ و میر ناشی از اسلحه می‌شود"]
predict_data_vectorized = DatasetBucket(predict_data, embedding_vectorizer, predict=True)

def pad_collate_fn_predict(batch):

    sequences_vectors, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), len(seq_vectors))
            for seq_vectors in sorted(batch, key=lambda x: len(x), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)
    pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

    return pack_padded_sequences_vectors


predict_loader = DataLoader(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
)
predictions = model.predict_generator(predict_loader, concatenate_returns=False)
predictions
idx_predictions = []
for batch in predictions:
    idx_predictions.extend(batch.argmax(axis=1).tolist()) 
idx_to_tags = {
     0:'<n>',
     1:'<c>',
     2:'<m>',
     3:'<e>',
     
          }

tags_predictions = []
for address in idx_predictions:
    tags_predictions.append([idx_to_tags.get(tag) for tag in address])

<ipython-input-47-dd92c7ae5f66>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), len(seq_vectors))


Prediction steps: 1 0.39s                                          


In [48]:
token = predict_data[0].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1

(راحت,<n>)
(به,<n>)
(دست,<c>)
(آوردن,<c>)
(اسلحه,<c>)
(منجر,<m>)
(به,<m>)
(نرخ,<e>)
(بالای,<e>)
(مرگ,<e>)
(و,<e>)
(میر,<e>)
(ناشی,<e>)
(از,<e>)
(اسلحه,<e>)
(می‌شود,<e>)


شبکه را دو لایه و دو طرفه می‌کنیم

In [35]:
dimension = 300
num_layer = 2
bidirectional = True

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

input_dim = dimension * 2  # since bidirectional

fully_connected_network = nn.Linear(input_dim, tag_dimension)

full_network_bi_lstm = RecurrentNet(lstm_network, fully_connected_network)

In [36]:
optimizer = optim.SGD(full_network_bi_lstm.parameters(), lr)
loss_function = nn.CrossEntropyLoss()

model2 = Model(
    full_network_bi_lstm,
    optimizer,
    loss_function,
     batch_metrics=['accuracy','f1' ,'recall'  ],
    device=device,
)

model2.fit_generator(train_loader, valid_loader, epochs=30)

test_loss, test_acc = model2.evaluate_generator(test_loader)


<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Step: 110/110 100.00% |████████████████████|ETA: 0.00s loss: 1.197742 acc: 48.396095 fscore_macro: 0.163064 recall_macro: 0.250000

<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch:  1/30 Train steps: 110 Val steps: 14 12.57s loss: 1.204488 acc: 50.315709 fscore_macro: 0.168192 recall_macro: 0.250061 val_loss: 1.176738 val_acc: 49.789264 val_fscore_macro: 0.166539 val_recall_macro: 0.250000


<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

...

Epoch: 30/30 Step: 108/110  98.18% |███████████████████▋|ETA: 0.12s loss: 0.963399 acc: 61.276333 fscore_macro: 0.416977 recall_macro: 0.416914

<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torc

Epoch: 30/30 Train steps: 110 Val steps: 14 9.50s loss: 0.949133 acc: 61.150544 fscore_macro: 0.421051 recall_macro: 0.423783 val_loss: 0.949691 val_acc: 60.864824 val_fscore_macro: 0.399405 val_recall_macro: 0.397765


<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-9-9c49fd1d320f>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Test steps: 14 1.57s test_loss: 0.949643 test_acc: 60.379900 test_fscore_macro: 0.409878 test_recall_macro: 0.408395                                                


و مجدد خروجی را چک می‌کنیم.

In [45]:
predict_data = ["راحت به دست آوردن اسلحه منجر به نرخ بالای مرگ و میر ناشی از اسلحه می‌شود"]
predict_data_vectorized = DatasetBucket(predict_data, embedding_vectorizer, predict=True)

def pad_collate_fn_predict(batch):

    sequences_vectors, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), len(seq_vectors))
            for seq_vectors in sorted(batch, key=lambda x: len(x), reverse=True)
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(sequences_vectors, batch_first=True, padding_value=0)
    pack_padded_sequences_vectors = pack_padded_sequence(padded_sequences_vectors, lengths.cpu(), batch_first=True)

    return pack_padded_sequences_vectors


predict_loader = DataLoader(
    predict_data_vectorized,
    batch_size=batch_size,
    collate_fn=pad_collate_fn_predict,
    num_workers=2,
)
predictions = model2.predict_generator(predict_loader, concatenate_returns=False)
predictions
idx_predictions = []
for batch in predictions:
    idx_predictions.extend(batch.argmax(axis=1).tolist()) 
idx_to_tags = {
     0:'<n>',
     1:'<c>',
     2:'<m>',
     3:'<e>',
     
          }

tags_predictions = []
for address in idx_predictions:
    tags_predictions.append([idx_to_tags.get(tag) for tag in address])

<ipython-input-45-535e40f38506>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), len(seq_vectors))


Prediction steps: 1 0.52s                                          


In [46]:
token = predict_data[0].split()
count = 0
for t in token :
  print( '(' + t + ',' + tags_predictions[0][count]+ ')')
  count +=1


(راحت,<c>)
(به,<c>)
(دست,<c>)
(آوردن,<c>)
(اسلحه,<c>)
(منجر,<c>)
(به,<c>)
(نرخ,<c>)
(بالای,<e>)
(مرگ,<e>)
(و,<e>)
(میر,<e>)
(ناشی,<e>)
(از,<e>)
(اسلحه,<e>)
(می‌شود,<n>)
